In [115]:
from ortools.sat.python import cp_model
import pandas as pd

In [116]:

def tamanho(modelo, estacios, avioes):
    avioes_grandes= [ aviao for aviao in avioes if aviao.grande]
    for estacio in estacios:
            for aviao in avioes_grandes:
                modelo.Add(estacio.variavel != aviao.k).OnlyEnforceIf(estacio.recebe_grande.Not())
  

def todos_estacionar(total_de_avioes, estacios, modelo):
    variaveis ={}
    for i in range(1,total_de_avioes+1):
        for j, estacio in enumerate(estacios):
            aviao_i_em_j = modelo.NewBoolVar(f"aviao_{i}_em_{j}")
            modelo.Add(estacio.variavel == i).OnlyEnforceIf(aviao_i_em_j)
            modelo.Add(estacio.variavel != i).OnlyEnforceIf(aviao_i_em_j.Not())
            variaveis[(i,j)] = aviao_i_em_j

    for i in range(1,total_de_avioes+1):
        modelo.AddExactlyOne([variaveis[(i,j)] for j in range(len(estacios))])    


def destintos(estacios, modelo):
    variaveis = [estacio.variavel for estacio in estacios]
    modelo.AddAllDifferent(variaveis)

def requerem_passaporte(modelo, estacios, avioes):
    avioes_com = [aviao for aviao in avioes if aviao.requer_passaporte]
    estacio_sem = [estacio for estacio in estacios if not estacio.tem_passaporte]
    for estacio in estacio_sem:
        for aviao in avioes_com:
            modelo.Add(estacio.variavel != aviao.k)


def prefere_com_passporte(modelo, estacios, avioes):
    penalidades =[]

    estacio_controle =[estacio for estacio in estacios if estacio.tem_passaporte]
    avioes_sem_controle = [ aviao for aviao in avioes if not aviao.requer_passaporte]
    for estacio in estacio_controle:
        for aviao in avioes_sem_controle:
            penalidade= modelo.NewIntVar(0, 1000, f"penalidade_{estacio.k}_{aviao.k}")

            aviao_estacionado= modelo.NewBoolVar(f"estacionamento_{estacio.k}_tem_aviao{aviao.k}")
            modelo.Add(estacio.variavel == aviao.k).OnlyEnforceIf(aviao_estacionado)
            modelo.Add(estacio.variavel != aviao.k).OnlyEnforceIf(aviao_estacionado.Not())

            modelo.Add(penalidade == 1000).OnlyEnforceIf(aviao_estacionado)
            modelo.Add(penalidade == 0).OnlyEnforceIf(aviao_estacionado.Not())
            penalidades.append(penalidade)
    return penalidades        



def limita_vizinhos(modelo, estacios, avioes):
    for estacio in estacios:
        if not estacio.grande:
            continue
        for vizinho in estacio.vizinhos:
            if vizinho.grande:
                modelo.Add(estacio.recebe_grande == 0).OnlyEnforceIf(vizinho.recebe_grande)


def resolver(solucionador, modelo, estacios, avioes, penalidades):
    modelo.Minimize(sum(penalidades))
    status = solucionador.Solve(modelo)
    print(solucionador.StatusName(status))
    if status == cp_model.INFEASIBLE:
        print("Não é possível resolver o problema")
        return
    
    print(f"Valor da função objetivo:{solucionador.ObjectiveValue()} ")

    for estacio in estacios:
        variavel = estacio.variavel
        valor = solucionador.Value(variavel)
        if valor == 0:
            print(f"Estacionamento {variavel} sem aviao")
        else:
            aviao= avioes[valor-1]
            print(f"Estacionamento {variavel} tem aviao {valor} grande = {aviao.grande}")


In [117]:
class estacio:
    def __init__(self, k, total_de_avioes, grande, modelo, tem_passaporte):
        self.grande = grande
        self.tem_passaporte = tem_passaporte 
        self.variavel = modelo.NewIntVar(0, total_de_avioes, f"estacionamento_{k}")
        self.k= k
        self.vizinhos = []
        self.recebe_grande = modelo.NewBoolVar(f"recebe_grande_{k}")
        if not self.grande:
            modelo.Add(self.recebe_grande == 0)
class aviao:
    def __init__(self, k, grande, requer_passaporte):
        self.k = k
        self.grande = grande    
        self.requer_passaporte = requer_passaporte

In [118]:
avioes = [aviao(1, False, True),
          aviao(2, True, False),
          aviao(3, False, False),
          aviao(4, True, True)]

modelo = cp_model.CpModel()
total_de_avioes = len(avioes)

estacios= [estacio(1, total_de_avioes, True, modelo, False),
           estacio(2, total_de_avioes, False, modelo, True),
           estacio(3, total_de_avioes, False, modelo, False),
           estacio(4, total_de_avioes, True, modelo,True)]
estacios[2].vizinhos = [estacios[3]]

destintos(estacios,modelo)
todos_estacionar(total_de_avioes, estacios, modelo)
limita_vizinhos(modelo, estacios, avioes)
tamanho(modelo, estacios, avioes)
requerem_passaporte(modelo, estacios, avioes)
penalidades = prefere_com_passporte(modelo, estacios, avioes)


solucionador = cp_model.CpSolver()
resolver(solucionador, modelo, estacios, avioes, penalidades)

OPTIMAL
Valor da função objetivo:0.0 
Estacionamento estacionamento_1 tem aviao 2 grande = True
Estacionamento estacionamento_2 tem aviao 1 grande = False
Estacionamento estacionamento_3 tem aviao 3 grande = False
Estacionamento estacionamento_4 tem aviao 4 grande = True
